In [3]:
import torch 
import h5py
import torchvision.transforms as transforms
import numpy as np

def get_transform():
    list_transforms = [
        transforms.ToTensor(),
    ]

    return list_transforms


class ToTensor(object):
    
    def __call__(self, sample):
        video_info, label = sample
        
        transform = get_transform()
        transform_video = transforms.Compose(transform)
        
        return np.array(video_info), np.array(label)


class VSMDataset(torch.utils.data.Dataset):
    """Video Summarizer Dataset
        Datasets: TVSum, Summe, VSUMM, CoSum, Visiocity
    """

    def __init__(self, hdfs_path, split=None, transform=None):
        """
        Args:
           hdfs_path (string): path of the hdfs processed data
           split (dict): idxs of the train/test split 
        """
        videos_info = h5py.File(hdfs_path)
        self.labels = {}
        self.data = {}
        self.transform = transform
        
        for it, video in enumerate(list(videos_info)):
            self.labels[it] = dict((key, videos_info[video][key][...])for key in list(videos_info[video]) if key in ('gtscore', 'gtsummary', 'user_summary') )
            self.data[it] = dict((key, videos_info[video][key][...])for key in list(videos_info[video]) if key not in ('gtscore', 'gtsummary', 'user_summary') )   

        if split:
            #TO-DO
            pass
        
        self.labels = dict(self.labels)
        self.data = dict(self.data)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        X = self.data[index]
        y = self.labels[index]

        #if self.transform:
        #    X, y = self.transform((X, y))
        
        print(type(X))
        print(type(y))
        
        return X, y

def show_sample(idx, hdfs_path):

    vsm_dataset = VSMDataset(hdfs_path=hdfs_path)
    video_info, label = vsm_dataset[idx]

    return video_info, label



In [4]:
path_just_generated = "/home/shuaman/video_sm/video_summarization/src/eccv16_dataset_summe_google_pool5.h5"

In [5]:
path_data_preprocessd = "/data/shuaman/video_summarization/datasets/object_features/eccv16_dataset_summe_google_pool5.h5"

In [6]:
video_info, label = show_sample(24, path_data_preprocessd)

<class 'dict'>
<class 'dict'>


In [7]:
type(label)

dict

In [8]:
training_set = VSMDataset(path_just_generated, transform=ToTensor())

In [ ]:
len(training_set)

In [ ]:
len(training_set.labels)

In [ ]:
len(training_set.data)

In [ ]:
params = {'batch_size': 1,
          'num_workers': 2}
max_epochs = 100

In [ ]:
training_generator = torch.utils.data.DataLoader(training_set, **params)

In [ ]:
for epoch in range(max_epochs):
    for video_info, label in training_generator:
        print(len(video_info))
        print(len(label))
        break
    break